<a href="https://colab.research.google.com/github/JungMYEONG-jin/Stats_Project/blob/window/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten, Dropout, Input, Conv1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, SpatialDropout1D
from keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten, Dropout, Input, Conv1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, SpatialDropout1D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

!ls

train = pd.read_csv('/content/drive/MyDrive/nhn/news_train.csv')

In [ ]:
train

In [ ]:
train.drop({'n_id', 'date'}, axis=1, inplace=True)

In [ ]:
train['content'] = train['title'] + train['content']

In [ ]:
# 띄어쓰기 제거 및 [] () ? %&@이상 기호 제거
train['content'] = train['content'].str.replace(" ", "")

In [ ]:
train['content'] = train['content'].str.replace("[","").str.replace("]","")

In [ ]:
import re
train['content'] = train['content'].apply(lambda x : re.sub(r"[()]", "", x))



In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import spacing

print(spacing(train['content'][313]))

def make_space(texts):
  lst=[]
  for text in texts:
    res = spacing(text)
    lst.append(res)
    print(res)
  return lst


In [ ]:
space_lst = make_space(train['content'])

In [ ]:
train['content'] = train['content'].apply(lambda x : re.sub(r"[@$&]", "", x)) # 기호 제거해야 한스펠 사용  가능

In [ ]:
#train['content'] = train['content'].apply(lambda x : re.sub(r"<[^>]+>", "", x))
#train['content'] = train['content'].apply(lambda x : re.sub(r"[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\_\~\$\'\"]", "", x))
train['content'] = train['content'].apply(lambda x : re.sub(r"\t", "", x))
train['content'] = train['content'].apply(lambda x : re.sub(r"\n", "", x))
train['content'] = train['content'].apply(lambda x : re.sub(r"\d+", "", x))
# 스펠 체커 자꾸 특수문자 오류떠서 마침표 콤마 제외하고 다 제외시켜버리자


In [ ]:
def text_token_process(texts):
    
    stopwords = ['을', '를', '이', '가', '은', '는' , '데', '등','null']
    #mecab = Mecab() # 일단 미캡사용
    mecab = Mecab()
    token_lst = []
    
    for text in texts:
        token = mecab.morphs(text) # 형태소 변환
        token = [t for t in token if t not in stopwords or type(t) !=float] # 불용어 걸러주기
        token_lst.append(token)
    return token_lst, mecab



In [ ]:
def normalize(data):
    normal=[]
    for i in data:
        i = i.lower()
        normal.append(i)
    return normal

reg_con = normalize(train['content'])

#train.drop('title', axis=1, inplace = True)
train['content'] = reg_con

train['new_content'], mecab = text_token_process(train['content'])

In [ ]:
train.drop('title', axis=1, inplace = True)

In [ ]:
y = train['info'] # target설정



y


maxlen = sum(len(i) for i in train['new_content'])
print(maxlen/len(train['new_content']))
print(len(train['new_content'][45644]))
train

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=1000):
    
    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len, padding='post') #설정한 문장의 최대 길이만큼 padding
    
    return X_train, vocab_size, tokenizer

train_X, vocab_size, vectorizer = text2sequence(train['new_content'], max_len = 200)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_X, y, test_size=0.2)

In [ ]:
model = Sequential([
    Embedding(vocab_size,32),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(16)),
    Dropout(0.5),
    Dense(64, activation='relu',kernel_regularizer = regularizers.l2(0.001)),
    Dense(1, activation='sigmoid')
])
# 이건 나중에 Bidirectional(LSTM(16)),
opt = Adam(learning_rate=0.01)
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])


model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# lr 001 아담
history = model.fit(x_train, y_train, epochs=10,validation_split=0.1, batch_size=90, callbacks=[early_stop])

In [ ]:
from keras.models import load_model

model.save('model_token_200.h5')

In [ ]:

test = pd.read_csv('/home/dktlsk6/Downloads/데이터분석/open/news_test.csv')

test['content'] = test['title']+test['content']

test.drop({'n_id', 'date', 'id', 'title'}, axis=1, inplace=True)


In [ ]:
reg_con = normalize(test['content'])

#train.drop('title', axis=1, inplace = True)
test['content'] = reg_con
test['content'] = test['content'].str.replace("[","").str.replace("]","")
test['content'] = test['content'].apply(lambda x : re.sub(r"[()]", "", x))
test['content'] = test['content'].apply(lambda x : re.sub(r"[@$&]", "", x))
#test['content'] = test['content'].apply(lambda x : re.sub(r"<[^>]+>", "", x))
#test['content'] = test['content'].apply(lambda x : re.sub(r"[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\_\~\$\'\"]", "", x))
test['content'] = test['content'].apply(lambda x : re.sub(r"\t", "", x))
test['content'] = test['content'].apply(lambda x : re.sub(r"\n", "", x))
test['content'] = test['content'].apply(lambda x : re.sub(r"\d+", "", x))

In [ ]:
test

In [ ]:
# 토큰화

test['new_content'], mecab = text_token_process(test['content'])

In [ ]:
test_X_seq = vectorizer.texts_to_sequences(test['new_content'])

X_test = pad_sequences(test_X_seq, maxlen=200, padding='post')


print(X_test.shape)

In [ ]:
pred = model.predict_classes(X_test)

In [ ]:
print('진짜뉴스 ', len(pred[pred==0]))
print('가짜뉴스 ', len(pred[pred==1]))

In [ ]:
sample = pd.read_csv('/home/dktlsk6/Downloads/데이터분석/open/sample_submission.csv')
sample['info'] = pred
sample.to_csv('test_13.csv', index=False)